In [ ]:
from eose.instruments import  CircularGeometry, RectangularGeometry, BasicSensor

from pydantic import ValidationError

# Example usage of SphericalGeometry class used to define the field of view of the instrument

circ_geom = CircularGeometry(diameter=90.0)
print(circ_geom)

rect_geom = RectangularGeometry(angle_height= 90.0, angle_width= 45)
print(rect_geom)


In [ ]:
# Example simple usage of eose-api BasicSensor class

sensor = BasicSensor(   mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        field_of_view = CircularGeometry(diameter=30.0),
                        data_rate= 10.5,
                        bits_per_pixel= 16
                    )

print(sensor)

sensor = BasicSensor(   mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        orientation= list([0.5,0.5,0.5,0.5]),
                        field_of_view = RectangularGeometry(ngle_width=30.0, angle_height=10.0),
                        data_rate= 50.5,
                        bits_per_pixel= 8
                    )

print(sensor)

sensor = BasicSensor(   mass= 100.5,
                    volume= 0.75,
                    power= 150.0,
                    orientation= list([0.5,0.5,0.5,0.5]),
                    data_rate= 50.5,
                    bits_per_pixel= 8
                )

print(sensor)

display(sensor.model_dump_json())

## Example usage with TAT-C and InstruPy

In [ ]:
import json
from datetime import datetime, timedelta, timezone

from instrupy.basic_sensor_model import BasicSensorModel

from tatc.schemas import Instrument, Satellite as TATC_Satellite, TwoLineElements, Point
from tatc.analysis import collect_orbit_track, OrbitCoordinate, OrbitOutput
from tatc.analysis import (
    collect_multi_observations,
    aggregate_observations,
    reduce_observations,
)

from eose.datametric import DataMetricRequest, BasicSensorDataMetricResponse
from eose.propagation import PropagationRecord, PropagationRequest, PropagationResponse
from eose.orbits import GeneralPerturbationsOrbitState
from eose.utils import CartesianReferenceFrame, FixedOrientation
from eose.coverage import (
    CoverageRecord,
    CoverageRequest,
    CoverageResponse,
    CoverageSample,
)
from eose.grids import UniformAngularGrid
from eose.satellites import Satellite

from shapely.geometry import box, mapping
from joblib import Parallel, delayed
from scipy.stats import hmean
import pandas as pd



In [ ]:
# Run propagation with TAT-C to get satellite states
def propagate_tatc(request: PropagationRequest) -> PropagationResponse:
    return PropagationResponse(
        records=collect_orbit_track(
            TATC_Satellite(
                name=request.orbit.object_name,
                orbit=TwoLineElements(tle=request.orbit.to_tle()),
            ),
            Instrument(name="Instrument"),
            pd.date_range(
                request.start, request.start + request.duration, freq=request.time_step
            ),
            coordinates=(
                OrbitCoordinate.ECI
                if request.frame == CartesianReferenceFrame.ICRF
                else OrbitCoordinate.ECEF
            ),
            orbit_output=OrbitOutput.POSITION_VELOCITY,
        ).apply(
            lambda r: PropagationRecord(
                time=r.time,
                frame=request.frame,
                position=r.geometry.coords[0],
                velocity=r.velocity.coords[0],
                body_orientation=FixedOrientation.NADIR_GEOCENTRIC,
            ),
            axis=1,
        ),
    )

iss_omm_str = '[{"OBJECT_NAME":"ISS (ZARYA)","OBJECT_ID":"1998-067A","EPOCH":"2024-06-07T09:53:34.728000","MEAN_MOTION":15.50975122,"ECCENTRICITY":0.0005669,"INCLINATION":51.6419,"RA_OF_ASC_NODE":3.7199,"ARG_OF_PERICENTER":284.672,"MEAN_ANOMALY":139.0837,"EPHEMERIS_TYPE":0,"CLASSIFICATION_TYPE":"U","NORAD_CAT_ID":25544,"ELEMENT_SET_NO":999,"REV_AT_EPOCH":45703,"BSTAR":0.00033759,"MEAN_MOTION_DOT":0.00019541,"MEAN_MOTION_DDOT":0}]'
iss_omm = json.loads(iss_omm_str)[0]

request = PropagationRequest(
    orbit=GeneralPerturbationsOrbitState.from_omm(iss_omm),
    start=datetime(2024, 1, 1, tzinfo=timezone.utc),
    duration=timedelta(hours=1),
    time_step=timedelta(minutes=1),
    frame=CartesianReferenceFrame.ICRF,
)

display(request.model_dump_json())

response = propagate_tatc(request)

display(response.model_dump_json())

data = response.as_dataframe()

display(data)


In [ ]:
# Run coverage analysis with TAT-C to get access-times at Target ground points

def coverage_tatc(request: CoverageRequest) -> CoverageResponse:
    unique_ids = len(set(target.id for target in request.targets)) == len(
        request.targets
    )
    points = [
        Point(
            id=target.id if unique_ids and isinstance(target.id, int) else i,
            longitude=target.position[0],
            latitude=target.position[1],
            altitude=target.position[2] if len(target.position) > 2 else 0,
        )
        for i, target in enumerate(request.targets)
    ]
    satellites = [
        TATC_Satellite(
            name=satellite.orbit.object_name,
            orbit=TwoLineElements(tle=satellite.orbit.to_tle()),
            instruments=[
                Instrument(name="Default", field_of_regard=satellite.field_of_view)
            ],
        )
        for satellite in request.satellites
    ]

    aggregated_obs = aggregate_observations(
        pd.concat(
            Parallel(-1)(
                delayed(collect_multi_observations)(
                    point,
                    satellites,
                    request.start,
                    request.start + request.duration,
                )
                for point in points
            )
        )
    )
    reduced_obs = reduce_observations(aggregated_obs)

    records = list(
        reduced_obs.apply(
            lambda r: CoverageRecord(
                target=request.targets[
                    points.index(next(p for p in points if p.id == r["point_id"]))
                ],
                samples=aggregated_obs[aggregated_obs.point_id == r["point_id"]].apply(
                    lambda s: CoverageSample(start=s.start, duration=s.end - s.start),
                    axis=1,
                ),
                mean_revisit=(
                    None
                    if pd.isnull(r["revisit"])
                    else timedelta(seconds=r["revisit"].total_seconds())
                ),
                number_samples=r["samples"],
            ),
            axis=1,
        )
    ) + [
        CoverageRecord(target=request.targets[i], mean_revisit=None, number_samples=0)
        for i, point in enumerate(points)
        if not any(reduced_obs["point_id"] == point.id)
    ]
    records.sort(key=lambda r: r.target.id)

    return CoverageResponse(
        records=records,
        harmonic_mean_revisit=(
            None
            if reduced_obs.dropna(subset="revisit").empty
            else timedelta(
                seconds=hmean(
                    reduced_obs.dropna(subset="revisit")["revisit"].dt.total_seconds()
                )
            )
        ),
        coverage_fraction=len(reduced_obs.index) / len(points),
    )



request = CoverageRequest(
    satellites=[
        Satellite(
            orbit=GeneralPerturbationsOrbitState.from_omm(iss_omm), field_of_view=100
        )
    ],
    targets=UniformAngularGrid(
        delta_latitude=20, delta_longitude=20, region=mapping(box(-180, -50, 180, 50))
    ).as_targets(),
    start=datetime(2024, 1, 1, tzinfo=timezone.utc),
    duration=timedelta(days=1),
)

display(request.model_dump_json())

response = coverage_tatc(request)

display(response.model_dump_json())

data = response.as_dataframe()

display(data)

In [ ]:
# Run InstruPy Basic Sensor datametrics calculator to get the incidence angle


def datametrics_instrupy(request: DataMetricRequest) -> BasicSensorDataMetricResponse:
    
    
    # Filter propagation record for each access and compute metrics throughout the access at each propagation point 


    return